<a href="https://colab.research.google.com/github/meln-ds/advent-of-code-2020/blob/main/AoC_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **--- Day 2: Password Philosophy ---**
Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via **toboggan**.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:

1-3 a: abcde  
1-3 b: cdefg  
2-9 c: ccccccccc  

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

**How many passwords are valid** according to their policies?

In [7]:
from google.colab import files
uploaded = files.upload()

Saving aoc_day2.txt to aoc_day2.txt


In [50]:
import io
import pandas as pd
import numpy as np

df = pd.read_csv(io.BytesIO(uploaded['aoc_day2.txt']), header=None)
df.head()

,0
0,6-8 s: svsssszslpsp
1,3-4 n: gncn
2,4-8 v: vkvmvdmvhttvvrgvvwv
3,16-18 j: jjjjjjjjjjjjjjjjjf
4,13-15 p: pppppppvppppxxppp


In [52]:
df = df[0].str.extractall('(\d+)\-(\d+)\ ([a-z])\:\ (.*)')
df = df.reset_index().drop(columns=['level_0','match'])
df[0] = df[0].astype('int')
df[1] = df[1].astype('int')

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1000 non-null   int64 
 1   1       1000 non-null   int64 
 2   2       1000 non-null   object
 3   3       1000 non-null   object
dtypes: int64(2), object(2)
memory usage: 31.4+ KB


In [54]:
df.head()

,0,1,2,3
0,6,8,s,svsssszslpsp
1,3,4,n,gncn
2,4,8,v,vkvmvdmvhttvvrgvvwv
3,16,18,j,jjjjjjjjjjjjjjjjjf
4,13,15,p,pppppppvppppxxppp


In [55]:
def password_match(row):
  count = row[3].count(row[2])
  if count >= row[0] and count <= row[1]:
    return 1
  else:
    return 0

In [56]:
df['match'] = df.apply(password_match, axis=1)

In [57]:
df['match'].sum()

515

In [58]:
df.head()

,0,1,2,3,match
0,6,8,s,svsssszslpsp,1
1,3,4,n,gncn,0
2,4,8,v,vkvmvdmvhttvvrgvvwv,0
3,16,18,j,jjjjjjjjjjjjjjjjjf,1
4,13,15,p,pppppppvppppxxppp,1


### **--- Part Two ---**
While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

1-3 a: abcde is valid: position 1 contains a and position 3 does not.  
1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.  
2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.  
How many passwords are valid according to the new interpretation of the policies?

In [59]:
def password_position_match(row):
  first_pos = row[0] - 1
  second_pos = row[1] - 1
  if row[3][first_pos] == row[2] and row[3][second_pos] == row[2]:
    return 0
  elif row[3][first_pos] != row[2] and row[3][second_pos] != row[2]:
    return 0
  else: return 1

In [60]:
df['pos_match'] = df.apply(password_position_match, axis=1)
df['pos_match'].sum()

711